## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

## 建立模型

In [2]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[28*28, 100],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
    def __call__(self, x):
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [3]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.1*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [4]:
train_data, test_data = mnist_dataset()
for epoch in range(20):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.2948396 ; accuracy 0.12088333
epoch 1 : loss 2.2474918 ; accuracy 0.15478334
epoch 2 : loss 2.2019415 ; accuracy 0.20815
epoch 3 : loss 2.1578164 ; accuracy 0.27623335
epoch 4 : loss 2.1148276 ; accuracy 0.34605
epoch 5 : loss 2.0727513 ; accuracy 0.40905
epoch 6 : loss 2.031416 ; accuracy 0.4635
epoch 7 : loss 1.9906956 ; accuracy 0.50875
epoch 8 : loss 1.9505026 ; accuracy 0.5485333
epoch 9 : loss 1.9107804 ; accuracy 0.57818335
epoch 10 : loss 1.8715011 ; accuracy 0.60431665
epoch 11 : loss 1.8326586 ; accuracy 0.6260667
epoch 12 : loss 1.7942669 ; accuracy 0.6446
epoch 13 : loss 1.7563541 ; accuracy 0.6599
epoch 14 : loss 1.7189615 ; accuracy 0.67285
epoch 15 : loss 1.6821369 ; accuracy 0.683
epoch 16 : loss 1.6459343 ; accuracy 0.6922
epoch 17 : loss 1.6104094 ; accuracy 0.6997
epoch 18 : loss 1.5756164 ; accuracy 0.70706666
epoch 19 : loss 1.5416073 ; accuracy 0.71313334
test loss 1.4929236 ; accuracy 0.7278
